In [ ]:
!pip install pillow
!pip install numpy


These commands use the pip package manager to install two Python packages. The first command installs the "Pillow" library, which is a powerful image processing library. The second command installs "NumPy," a fundamental package for scientific computing with Python, providing support for large, multi-dimensional arrays and matrices.

While seeking ways to enhance the accuracy of our existing models, I came across an article highlighting the challenge of recognizing sequential printed texts that have become blurred and noisy over time due to repeated use. Intrigued by this issue, I explored models capable of addressing such challenges and discovered Hidden Markov Models (HMMs), which leverage probabilities for recognition. Upon implementing HMMs, I observed notable improvements in text recognition for degraded and distorted printed texts.

In [ ]:
from __future__ import division
from PIL import Image, ImageDraw, ImageFont
import sys
import math
import numpy as np
CHARACTER_WIDTH=14
CHARACTER_HEIGHT=25
TRAIN_LETTERS="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789(),.-!?\"' "

def load_letters(fname):
    im = Image.open(fname)
    px = im.load()
    (x_size, y_size) = im.size
    print(im.size)
    print(int(x_size / CHARACTER_WIDTH) * CHARACTER_WIDTH)
    result = []
    for x_beg in range(0, int(x_size / CHARACTER_WIDTH) * CHARACTER_WIDTH, CHARACTER_WIDTH):
        result += [ [ "".join([ '*' if px[x, y] < 1 else ' ' for x in range(x_beg, x_beg+CHARACTER_WIDTH) ]) for y in range(0, CHARACTER_HEIGHT) ], ]
    return result

def load_training_letters(fname):
    TRAIN_LETTERS="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789(),.-!?\"' "
    letter_images = load_letters(fname)
    return { TRAIN_LETTERS[i]: letter_images[i] for i in range(0, len(TRAIN_LETTERS) ) }

This code defines functions for loading training letters from an image file. It utilizes the Pillow (PIL) library for image processing. The load_letters function takes an image file, converts it into a pixel matrix, and extracts character-sized blocks, storing them as binary strings in a list. The load_training_letters function maps these binary strings to corresponding characters in the TRAIN_LETTERS string, creating a dictionary where each character is associated with its respective pixel representation.

In [ ]:
def init_trans(filename):
    train_txt = open("/kaggle/input/ntguyf-7/12345.txt",'r')
    lines = train_txt.readlines()

    file_temp = []
    word_temp=[]
    for line in lines:
        for word in line.split():
            word_temp.append(word)
        for a in line:
            file_temp.append(a)
    init_temp = [0]*len(TRAIN_LETTERS)

    for a in range(len(TRAIN_LETTERS)):
        for w in word_temp:
            if TRAIN_LETTERS[a] == w[0]:
                init_temp[a] +=1

        for a in line:
            file_temp.append(a)
    train_txt.close()
    init = [(i+1)/(len(word_temp)+2) for i in init_temp]
    trans = np.zeros(shape=(len(TRAIN_LETTERS), len(TRAIN_LETTERS)))
    for i in range(0,len(file_temp)):
        if file_temp[i] in TRAIN_LETTERS and file_temp[i+1] in TRAIN_LETTERS:
            t_index = TRAIN_LETTERS.index(file_temp[i])
            t1_index = TRAIN_LETTERS.index(file_temp[i+1])
            trans[t_index,t1_index] += 1
    rows_sum = np.sum(trans,axis=1)
    for i in range(0,len(TRAIN_LETTERS)):
        for j in range(0,len(TRAIN_LETTERS)):
            trans[i,j] = (trans[i,j] + 1)/(rows_sum[i]+2) #Laplace Smoothing

    return init,trans

def calc_emission_prob():
    noise = 0.42
    num_i = len(train_letters)
    num_j = len(test_letters)
    emissions = np.zeros(shape=(num_i,num_j))
    for letter in train_letters:
        for j in range(num_j):
            val = train_letters.get(letter)
            obs = test_letters[j]
            mis = 0
            match = 0
            for m in range(25):
                for n in range(14):
                    if val[m][n] != obs[m][n]:
                        mis += 1 #the number of mismatched pixels
                    else:
                        match += 1 #the number of matched pixels
            emissions[TRAIN_LETTERS.index(letter)][j] = (math.pow(1-noise,match)) * (math.pow(noise,mis))

    return emissions

This code defines two functions for initializing transition probabilities and calculating emission probabilities for a Hidden Markov Model (HMM). The init_trans function computes the initial state probabilities (init) and transition probabilities (trans) between letters in the TRAIN_LETTERS set. It uses Laplace Smoothing to handle cases where certain transitions have zero occurrences. The calc_emission_prob function calculates emission probabilities based on pixel comparisons between training and test letters, incorporating a noise factor. It computes the likelihood of observing a test letter given a training letter and returns a matrix of emission probabilities (emissions). The code is likely part of a broader HMM-based pattern recognition or classification system.

In [ ]:
def simplified(test_letters,init,ems):
    init =init
    emission = ems
    obs = test_letters
    rows = len(TRAIN_LETTERS)
    cols = len(obs)
    y = np.zeros(cols)
    temp_results = np.zeros(shape=(rows,cols))
    for i in range(0,cols):
        for j in range(0,rows):
            temp_results[j,i] = emission[j,i]
    y = np.argmax(temp_results, axis=0)
    return y
def hmm_ve(test_letters,init,ems,trans):
    init =init
    emission = ems
    transition = trans
    obs = test_letters
    rows = len(TRAIN_LETTERS)
    cols = len(obs)
    out = np.zeros(shape=(rows,cols))
    for i in range(rows):
        out[i,0] = init[i] * emission[i,0]
    for i in range(cols):
        for j in range(rows):
            temp =0
            for k in range(rows):
                temp += init[k]*transition[k,i]
            out[j,i] = temp* emission[j,i]
    indexes = np.argmax(out,axis=0)
    return indexes
def viterbi(test_letters,init,trans,ems):
    obs = test_letters
    init = init
    transition = trans
    emission = ems
    t = len(obs)
    rows = len(train_letters)
    v_it = np.zeros(shape=(rows,t))
    max_path =  np.empty(shape=(rows,t),dtype=int)
    for i in range(rows):
        v_it[i,0] = math.log(init[i]) + math.log(emission[i,0])
    for j in range(1,t):
        for i in range(rows):
            temp_val = []
            for k in range(rows):
                temp_val.append(v_it[k,j-1] + math.log(transition[k,i] ) +math.log(emission[i,j]))
            max_state = max(temp_val)
            v_it[i,j] = max_state
            max_path[i,j] = temp_val.index(max_state)
    most_likely = np.zeros(t,dtype=int)
    most_likely[-1] = np.argmax(v_it[:,-1])
    for i in range(1,t)[::-1]:
        most_likely[i-1] = max_path[most_likely[i],i]
    return most_likely

simplified: This function performs the simplified HMM algorithm, where for each observation in test_letters, it selects the state with the highest emission probability from the matrix ems. The resulting sequence of state indices is returned.

hmm_ve: This function implements the HMM algorithm using the Viterbi algorithm. It computes the most likely state sequence for the given observations in test_letters based on the initial probabilities (init), transition probabilities (trans), and emission probabilities (ems).

viterbi: This function is a more detailed implementation of the Viterbi algorithm. It calculates the log probabilities of all possible state sequences and efficiently finds the most likely sequence given the observations in test_letters, initial probabilities (init), transition probabilities (trans), and emission probabilities (ems).

In [ ]:
train_img_fname = "/kaggle/input/ntguyf-7/courier-train.png"
train_txt_fname = "/kaggle/input/ntguyf-7/12345.txt"
test_img_fname = "/kaggle/input/iuasdfh/test-5-0.png"
train_letters = load_training_letters(train_img_fname)
test_letters = load_letters(test_img_fname)
init, trans = init_trans(train_txt_fname)
ems = calc_emission_prob()
simple = simplified(test_letters,init,ems)
print ("Simple: "+"".join([TRAIN_LETTERS[i] for i in simple]))
hmm_out = hmm_ve(test_letters,init,ems,trans)
print ("HMM VE: " + "".join([TRAIN_LETTERS[i] for i in hmm_out]))
output = viterbi(test_letters,init,trans,ems)
print ("HMM MAP: " + "".join([TRAIN_LETTERS[i] for i in output]))

(1008, 25)
1008
(281, 25)
280
Simple: Opinion of the Ccurt
HMM VE: Opinion of the Ccurt
HMM MAP: Opinion of the Court


In [ ]:
train_img_fname = "/kaggle/input/ntguyf-7/courier-train.png"
train_txt_fname = "/kaggle/input/ntguyf-7/12345.txt"
test_img_fname = "/kaggle/input/new-data/test-15-0.png"
train_letters = load_training_letters(train_img_fname)
test_letters = load_letters(test_img_fname)
init, trans = init_trans(train_txt_fname)
ems = calc_emission_prob()
simple = simplified(test_letters,init,ems)
print ("Simple: "+"".join([TRAIN_LETTERS[i] for i in simple]))
hmm_out = hmm_ve(test_letters,init,ems,trans)
print ("HMM VE: " + "".join([TRAIN_LETTERS[i] for i in hmm_out]))
output = viterbi(test_letters,init,trans,ems)
print ("HMM MAP: " + "".join([TRAIN_LETTERS[i] for i in output]))

(1008, 25)
1008
(561, 25)
560
Simple: The Constitut1on grants them that r1ght.
HMM VE: The Constitut1on grants them that r1ght.
HMM MAP: The Constitution grants them that right.


In [ ]:
train_img_fname = "/kaggle/input/ntguyf-7/courier-train.png"
train_txt_fname = "/kaggle/input/ntguyf-7/12345.txt"
test_img_fname = "/kaggle/input/new-data/test-17-0.png"
train_letters = load_training_letters(train_img_fname)
test_letters = load_letters(test_img_fname)
init, trans = init_trans(train_txt_fname)
ems = calc_emission_prob()
simple = simplified(test_letters,init,ems)
print ("Simple: "+"".join([TRAIN_LETTERS[i] for i in simple]))
hmm_out = hmm_ve(test_letters,init,ems,trans)
print ("HMM VE: " + "".join([TRAIN_LETTERS[i] for i in hmm_out]))
output = viterbi(test_letters,init,trans,ems)
print ("HMM MAP: " + "".join([TRAIN_LETTERS[i] for i in output]))

(1008, 25)
1008
(239, 25)
238
Simple: 1t 1s so orcerec.
HMM VE: 1t 1s so orcerec.
HMM MAP: It is so ordered.
